In [ ]:
import json
import pickle
import os.path
import time
import csv
import random
import datetime
import sys
import pymongo
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
from os import path
from pprint import pprint

suspended_users = []
normal_users = []

In [ ]:
# all these are of 29th June
# with open('../useddata/deleted_ids.txt','r') as f:
#     for users in f:
#         deleted_users.append(users)
        
with open('../data/suspended_ids.txt','r') as f:
    for users in f:
        suspended_users.append(users.strip())
        
with open('../data/normal_account_ids.txt','r') as f: #id_final is the list of all unique users.
    for users in f:
        normal_users.append(users.strip())

In [ ]:
print(len(suspended_users) , len(normal_users)) 

In [ ]:
# all_users = deleted_users + suspended_users + normal_users
# print(len(all_users))

In [ ]:
# all_users = [userid.strip() for userid in all_users]
sus_ids_dic = {}
nor_ids_dic = {}

In [ ]:
# querying_mongodb for tweets by people belonging to the requisite list
USERNAME = ""
PASSWORD = ""

# connection for storing tweets
client = MongoClient('192.168.29.151',
                     username=USERNAME,
                     password=PASSWORD,
                     authSource='elections2019',
                     authMechanism='SCRAM-SHA-1')
db = client.elections2019

suspended_list_tweets = []
normal_list_tweets = []

i = 0
for collection in tqdm(db.list_collection_names(), desc='collections_done'):
    i+=1
#     if collection != 'tweets':
#         continue
#     print(collection)
#     print(db[collection].find_one())
#     break
    query = {}

    normal_tweets = []
    
    projection = {}
    projection["_id"] = 0

# User parameters stored
    projection["user"] = 1
    
# Tweet parameters stored    
    projection["id_str"] = 1
    projection["quote_count"] = 1
    projection["reply_count"] = 1
    projection["retweet_count"] = 1
    projection["favorite_count"] = 1
    projection["created_at"] = 1
    projection["entities"] = 1
    projection["retweeted_status.id_str"] = 1
    projection["retweeted_status.user.id_str"] = 1
    projection["retweeted_status.full_text"] = 1
    projection["text"] = 1
    projection['retweeted_status.extended_tweet.full_text'] = 1
    projection['extended_tweet.full_text'] = 1
    projection['full_text'] = 1
    projection['retweeted_status.text'] = 1
     
    cursor = db[collection].find( { "user.id_str": { "$in": suspended_users } } , projection = projection)
    for tweets in tqdm(cursor):
        if tweets["id_str"] in sus_ids_dic:
            continue
        else:
            sus_ids_dic[tweets["id_str"]] = 1
            
        suspended_list_tweets.append(tweets)
        if(len(suspended_list_tweets) >= 50000):
            with open('../data/all_tweets_suspended_jan3_2021.pickle', 'ab+') as fp:
                pickle.dump(suspended_list_tweets,fp)
            suspended_list_tweets = []
    print(len(suspended_list_tweets))
    with open('../data/all_tweets_suspended_jan3_2021.pickle', 'ab+') as fp:
        pickle.dump(suspended_list_tweets,fp)
    
    suspended_list_tweets = []
    cursor.close()
    
    cursor = db[collection].find( { "user.id_str": { "$in": normal_users } } , projection = projection)
    for tweets in tqdm(cursor):
        if tweets["id_str"] in nor_ids_dic:
            continue
        else:
            nor_ids_dic[tweets["id_str"]] = 1
        
        normal_list_tweets.append(tweets)
        if(len(normal_list_tweets) >= 50000):
            with open('../data/all_tweets_normal_jan3_2021.pickle', 'ab+') as fp:
                pickle.dump(normal_list_tweets,fp)
            normal_list_tweets = []
    print(len(normal_list_tweets))
    with open('../data/all_tweets_normal_jan3_2021.pickle', 'ab+') as fp:
        pickle.dump(normal_list_tweets,fp)
    
    
    normal_list_tweets = []
    print(len(sus_ids_dic))
    print(len(nor_ids_dic))
    cursor.close()

In [ ]:
print(len(sus_ids_dic))
print(len(nor_ids_dic))